In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/the_data.csv")

In [3]:
df.head()

,Unnamed: 0,book_authors,book_desc,book_title,genres,image_url,book_desc_filt
0,0,['Suzanne Collins'],winning will make you famous. losing means cer...,The Hunger Games,youngadult fiction sciencefiction dystopia fan...,https://images.gr-assets.com/books/1447303603l...,win make famous . lose mean certain death.th n...
1,1,"['J.K. Rowling', 'Mary GrandPré']",there is a door at the end of a silent corrido...,Harry Potter and the Order of the Phoenix,fantasy youngadult fiction,https://images.gr-assets.com/books/1255614970l...,door end silent corridor . ’ haunt harri pottt...
2,2,['Harper Lee'],the unforgettable novel of a childhood in a sl...,To Kill a Mockingbird,classics fiction historical historicalfiction ...,https://images.gr-assets.com/books/1361975680l...,unforgett novel childhood sleepi southern town...
3,3,['Stephenie Meyer'],about three things i was absolutely positive.f...,Twilight,youngadult fantasy romance paranormal vampires...,https://images.gr-assets.com/books/1361039443l...,"three thing absolut positive.first , edward va..."
4,4,['Markus Zusak'],trying to make sense of the horrors of world w...,The Book Thief,historical historicalfiction fiction youngadult,https://images.gr-assets.com/books/1522157426l...,"tri make sens horror world war ii , death rela..."


In [4]:
for i in range(df.shape[0]):
    df["genres"][i] = (df["genres"][i]+" ")*int(len(df["book_desc_filt"][i])*0.009)

/tmp/ipykernel_849/2474743883.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["genres"][i] = (df["genres"][i]+" ")*int(len(df["book_desc_filt"][i])*0.009)


In [5]:
df["combined_features"] = df["genres"] + ["book_desc_filt"]

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
#df['genres_feature'] = ['genres_{}'.format(x) for x in df['genres']]

In [8]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["combined_features"])

In [9]:
cosine_sim = cosine_similarity(X, X[0])

In [10]:
cosine_sim.flatten()[1:].argmax()

253

In [11]:
df.iloc[[0,19610]]

,Unnamed: 0,book_authors,book_desc,book_title,genres,image_url,book_desc_filt,combined_features
0,0,['Suzanne Collins'],winning will make you famous. losing means cer...,The Hunger Games,youngadult fiction sciencefiction dystopia fan...,https://images.gr-assets.com/books/1447303603l...,win make famous . lose mean certain death.th n...,youngadult fiction sciencefiction dystopia fan...
19610,19610,['Marilyn Harris'],he was the power and the fury of a dynasty. sh...,The Prince of Eden,historical historicalfiction romance historica...,https://images.gr-assets.com/books/1277768727l...,power furi dynasti . love could destroy himedw...,historical historicalfiction romance historica...


In [12]:
df.book_title.unique()[250:300]

array(['The Guernsey Literary and Potato Peel Pie Society',
       'Catching Fire', 'The Green Mile', 'The House of the Spirits',
       'Delirium', "The Girl Who Kicked the Hornet's Nest",
       'I Know This Much Is True', 'White Oleander', 'The Vampire Lestat',
       'Beautiful Disaster', 'Paper Towns', 'Othello',
       "Lamb: The Gospel According to Biff, Christ's Childhood Pal",
       'The Once and Future King',
       'Flow Down Like Silver: Hypatia of Alexandria', 'Misery',
       'Into the Wild', 'King Lear', 'The Idiot', 'Speak',
       "Howl's Moving Castle", 'Cinder', 'Neverwhere', 'The Historian',
       "Uncle Tom's Cabin", 'Invisible Man', 'James and the Giant Peach',
       'Inkheart', 'All Creatures Great and Small', 'Leaves of Grass',
       'The Thirteenth Tale', 'Graceling', 'Candide',
       'The Cider House Rules', 'Paradise Lost',
       'The Things They Carried', 'Different Seasons',
       'The Complete Sherlock Holmes, Vol 1', 'Heidi', 'Foundation',
       "

In [13]:
libros = df[df["book_title"] == "Othello"]

In [14]:
libros

,Unnamed: 0,book_authors,book_desc,book_title,genres,image_url,book_desc_filt,combined_features
264,264,['William Shakespeare'],"in othello, shakespeare creates a powerful dra...",Othello,classics plays drama fiction academic school c...,https://images.gr-assets.com/books/1459795105l...,"othello , shakespear creat power drama marriag...",classics plays drama fiction academic school c...
45189,45189,['William Shakespeare'],"in othello, shakespeare creates a powerful dra...",Othello,classics plays drama fiction academic school c...,https://images.gr-assets.com/books/1384802673l...,"othello , shakespear creat power drama marriag...",classics plays drama fiction academic school c...


In [15]:
selected_books = vectorizer.transform(libros["combined_features"])

In [16]:
cosine_sim = cosine_similarity(X, selected_books)

In [17]:
cosine_sim.shape

(46016, 2)

In [18]:
recommend = df.copy()
recommend["sim"] = cosine_sim.sum(axis=-1)

In [24]:
recommend.sort_values("sim", ascending=False)[recommend["book_title"]!="Othello"].drop_duplicates(subset="book_title").head(15)

/tmp/ipykernel_849/3360549178.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  recommend.sort_values("sim", ascending=False)[recommend["book_title"]!="Othello"].drop_duplicates(subset="book_title").head(15)


,Unnamed: 0,book_authors,book_desc,book_title,genres,image_url,book_desc_filt,combined_features,sim
560,560,"['William Shakespeare', 'Roma Gill', 'فرنگیس ش...",the oxford school shakespeare has become the p...,Julius Caesar,classics plays fiction drama academic school c...,https://images.gr-assets.com/books/1354574927l...,oxford school shakespear becom prefer introduc...,classics plays fiction drama academic school c...,2.000000
39303,39303,['William Shakespeare'],undoubtedly the most famous of all of shakespe...,Hamlet,classics plays fiction drama academic school c...,https://images.gr-assets.com/books/1366931642l...,"undoubt famous shakespear 's play , hamlet rem...",classics plays fiction drama academic school c...,2.000000
6008,6008,"['Reginald Rose', 'David Mamet']",a landmark american drama that inspired a clas...,Twelve Angry Men,plays classics fiction drama academic school p...,https://images.gr-assets.com/books/1327910292l...,landmark american drama inspir classic film br...,plays classics fiction drama academic school p...,1.999999
27825,27825,"['Sophocles', 'Robert Fagles', 'Bernard Knox']",the heroic greek dramas that have moved theate...,The Three Theban Plays: Antigone / Oedipus the...,classics plays drama fiction academic school c...,https://images.gr-assets.com/books/1520346697l...,heroic greek drama move theatergo reader sinc ...,classics plays drama fiction academic school c...,1.999998
19800,19800,"['William Shakespeare', 'Cyrus Hoy']",\r\r\r\nthis revised norton critical edition o...,Hamlet (Norton Critical Edition),classics plays fiction drama academic school c...,https://images.gr-assets.com/books/1348564043l...,revis norton critic edit one seri ' wide read ...,classics plays fiction drama academic school c...,1.999996
38857,38857,"['William Shakespeare', 'Bernice W. Kliman', '...",george lyman kittredge’s insightful editions o...,The Tragedy of Hamlet (New Kitteredge Shakespe...,classics plays fiction drama academic school c...,https://images.gr-assets.com/books/1388540617l...,georg lyman kittredg ’ insight edit shakespear...,classics plays fiction drama academic school c...,1.999992
395,395,['Tennessee Williams'],the pulitzer prize and drama critics circle aw...,A Streetcar Named Desire,plays classics drama fiction academic school p...,https://images.gr-assets.com/books/1389153133l...,pulitz prize drama critic circl award win play...,plays classics drama fiction academic school p...,1.999961
41255,41255,"['Lorraine Hansberry', 'Robert Nemiroff']","""never before, in the entire history of the am...",A Raisin In The Sun,plays classics fiction drama academic school p...,https://images.gr-assets.com/books/1444920086l...,"`` never , entir histori american theater , mu...",plays classics fiction drama academic school p...,1.999961
595,595,"['Sophocles', 'J.E. Thomas']","""...what man wins more happiness than just its...",Oedipus Rex,classics plays drama fiction academic school c...,https://images.gr-assets.com/books/1388182316l...,`` ... man win happi shape ruin shape collaps ...,classics plays drama fiction academic school c...,1.999961
3064,3064,['Lorraine Hansberry'],"""never before, in the entire history of the am...",A Raisin in the Sun,plays classics fiction drama academic school p...,https://images.gr-assets.com/books/1165522672l...,"`` never , entir histori american theater , mu...",plays classics fiction drama academic school p...,1.999961
